In [18]:
def extended_gcd(a, mod):
    if a == 0:
        return mod, 1, 0
    gcd, x, y = extended_gcd(mod % a, a)
    c = x - (mod // a) * y
    return gcd, y, c

def modular_inverse(a, mod):
    gcd, y, c = extended_gcd(a, mod)
    answ = [gcd, y, c]
    return answ
def solve_linear_congruence(a, b, mod):
    answ = modular_inverse(a, mod)
    gcd = answ[0]

    if gcd == 1:
        c = answ[2]
        return b * c % mod
    elif gcd > 1 and b % gcd == 0:
        answ = modular_inverse(int(a / gcd), int(mod / gcd))
        gcd = answ[0]
        c = answ[2]
        multi = int(b / gcd * c % mod)
        return [multi + int(mod / gcd) * i for i in range(gcd)]
from collections import Counter

def top_character_bigrams_with_relative_frequency(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()
    text = ''.join(c.lower() for c in text if c.isalnum() or c.isspace())
    bigrams = [text[i:i+2] for i in range(len(text) - 1)]
    bigram_counts = Counter(bigrams)
    total_bigrams = sum(bigram_counts.values())
    top_5_bigrams = bigram_counts.most_common(5)

    for bigram, count in top_5_bigrams:
        relative_frequency = count / total_bigrams
        print(f"'{bigram}': {count} (частота: {relative_frequency:.2%})")

top_character_bigrams_with_relative_frequency('11_utf.txt')

'хб': 60 (частота: 0.77%)
'нк': 56 (частота: 0.72%)
'бй': 53 (частота: 0.68%)
'юж': 52 (частота: 0.67%)
'шь': 49 (частота: 0.63%)


In [19]:
cipher_bigrams = ['хб', 'нк', 'бй', 'юж', 'шь']
russian_bigrams = ['ст', 'но', 'то', 'на', 'ен']
alphabet = 'абвгдежзийклмнопрстуфхцчшщьыэюя'
letter_to_number = {letter: index for index, letter in enumerate(alphabet)}
m_squared = 961
pair_list = [(russian_bigram, cipher_bigram)
             for russian_bigram in russian_bigrams
             for cipher_bigram in cipher_bigrams]
print("Pair list:", pair_list)
numeric_pairs = [
    (
        31 * letter_to_number[pair[0][0]] + letter_to_number[pair[0][1]],
        31 * letter_to_number[pair[1][0]] + letter_to_number[pair[1][1]]
    )
    for pair in pair_list
]
print("Numeric pairs:", numeric_pairs)

keys = []
for i in range(len(numeric_pairs)):
    X1, Y1 = numeric_pairs[i]

    for j in range(i + 1, len(numeric_pairs)):
        X2, Y2 = numeric_pairs[j]
        delta_X = X1 - X2
        delta_Y = Y1 - Y2
        if delta_X == 0:
            continue

        try:
            answ = modular_inverse(delta_X, m_squared)
            inv_delta_X = answ[2]
            a = (delta_Y * inv_delta_X) % m_squared
            b = (Y1 - a * X1) % m_squared
            result = solve_linear_congruence(a, b, m_squared)
            if result is not None:
                keys.append((a, b))

        except ValueError:
            continue

print("\nFinal keys:")
for key in keys:
    print(key)

Pair list: [('ст', 'хб'), ('ст', 'нк'), ('ст', 'бй'), ('ст', 'юж'), ('ст', 'шь'), ('но', 'хб'), ('но', 'нк'), ('но', 'бй'), ('но', 'юж'), ('но', 'шь'), ('то', 'хб'), ('то', 'нк'), ('то', 'бй'), ('то', 'юж'), ('то', 'шь'), ('на', 'хб'), ('на', 'нк'), ('на', 'бй'), ('на', 'юж'), ('на', 'шь'), ('ен', 'хб'), ('ен', 'нк'), ('ен', 'бй'), ('ен', 'юж'), ('ен', 'шь')]
Numeric pairs: [(545, 652), (545, 413), (545, 40), (545, 905), (545, 770), (417, 652), (417, 413), (417, 40), (417, 905), (417, 770), (572, 652), (572, 413), (572, 40), (572, 905), (572, 770), (403, 652), (403, 413), (403, 40), (403, 905), (403, 770), (168, 652), (168, 413), (168, 40), (168, 905), (168, 770)]

Final keys:
(610, 708)
(215, 719)
(456, 70)
(885, 749)
(258, 348)
(343, 151)
(133, 242)
(138, 400)
(604, 134)
(654, 753)
(587, 750)
(703, 956)
(643, 21)
(203, 532)
(216, 174)
(925, 91)
(351, 357)
(566, 424)
(807, 736)
(275, 454)
(703, 717)
(85, 216)
(836, 307)
(841, 465)
(357, 931)
(50, 71)
(944, 68)
(99, 274)
(318, 83)
(521

In [20]:
char_to_num = {char: i for i, char in enumerate(alphabet)}
num_to_char = {i: char for i, char in enumerate(alphabet)}

with open('11_utf.txt', 'r', encoding='utf-8') as file:
    cipher_text = file.read()
    cipher_text = cipher_text.replace('\n', '').replace('\r', '')
cipher_nums = [char_to_num[char] for char in cipher_text]

def affine_decrypt(cipher_nums, a, b, mod):
    m_squared = mod ** 2
    decrypted_text = []

    try:
        answ = modular_inverse(a, m_squared)
        a_inv = answ[2]
    except ValueError:
        print(f"Оберненого не існує. Пропускаем ключ.")
        return None

    for i in range(0, len(cipher_nums), 2):
        y = cipher_nums[i]
        next_y = cipher_nums[i + 1] if i < len(cipher_nums) - 1 else 0
        ind = y*31 + next_y
        x = (a_inv * (ind - b)) % m_squared
        x1, x2 = (x // 31), (x%31)

        decrypted_text.append(num_to_char[x1 % mod])
        decrypted_text.append(num_to_char[x2 % mod])

    return ''.join(decrypted_text)

decrypted_texts = {}
for a, b in keys:
    result = affine_decrypt(cipher_nums, a, b, len(alphabet))
    if result:
        decrypted_texts[(a, b)] = result

In [21]:
rare_bigrams = ["щт", "ьо", "ыж", "юв", "яы", "аы", "бй", "гй", "дй", "еы", "шщ", "шя", "щб", "щд", "щж", "ьы", "ыа", "ыь", "ыы", "ыэ"]

def split_into_bigrams(text):
    return [text[i:i+2] for i in range(0, len(text), 2)]

def count_rare_bigrams(text, rare_bigrams):
    bigrams = split_into_bigrams(text)
    count = sum(1 for bigram in bigrams if bigram in rare_bigrams)
    return count

bigram_counts = {}
for key, decrypted_text in decrypted_texts.items():
    count = count_rare_bigrams(decrypted_text, rare_bigrams)
    bigram_counts[key] = count
sorted_bigram_counts = sorted(bigram_counts.items(), key=lambda x: x[1])
print("Top 10 ключів з найменшою кількістю рідкісних біграм:")
for i, (key, count) in enumerate(sorted_bigram_counts[:10]):
    print(f"{i+1}. Ключ {key} - Кількість рідкісних біграм: {count}")

Top 10 ключів з найменшою кількістю рідкісних біграм:
1. Ключ (703, 956) - Кількість рідкісних біграм: 5
2. Ключ (845, 564) - Кількість рідкісних біграм: 14
3. Ключ (50, 71) - Кількість рідкісних біграм: 22
4. Ключ (756, 289) - Кількість рідкісних біграм: 23
5. Ключ (36, 370) - Кількість рідкісних біграм: 23
6. Ключ (258, 348) - Кількість рідкісних біграм: 26
7. Ключ (925, 91) - Кількість рідкісних біграм: 26
8. Ключ (636, 236) - Кількість рідкісних біграм: 26
9. Ключ (885, 749) - Кількість рідкісних біграм: 27
10. Ключ (334, 843) - Кількість рідкісних біграм: 27


In [22]:
final_key = (703, 956)
print(decrypted_texts[final_key])

хорошосэрбиллнехотясунулденьгивкарманвотчтобиллвыпростопосеетеэтуновуютравукогданибудьвдругойразкактолькояпомрунадругойжеденьможетеперекопатьэтучертовулужайкунукакхватитувастерпенияподождатьещелетпятьшестьчтобыстарыйболтунуспелотдатьконцыужбудьтеувереныподождусказалбиллсамнезнаюкаквамобяснитьнодляменяжужжаньеэтойкосилкисамаяпрекраснаямелодиянасветевнейвсяпрелестьлетабезнееябыужаснотосковалибеззапахасвежескошеннойтравытожебиллнагнулсяиподнялсземликорзинкуяпошелковрагувыславныйюношаивсепонимаетеяуверенизвасполучитсяблестящийиумныйрепортерсказалдедушкапомогаяемуподнятькорзинкуявамэтопредсказываюпрошлоутронаступилполденьпослеобедадедушкаподнялсяксебенемногопочиталуиттиераикрепкоуснулкогдаонпроснулсябылотричасавокнавливалсяяркийивеселыйсолнечныйсветдедушкалежалвкроватиивдругвздрогнулслужайкидоносилосьпрежнеезнакомоенезабываемоежужжаньечтоэтосказалонктотокоситтравуноведьеетолькосегодняутромскосилионещепослушалдаконечноэтожужжиткосилкамернонеутомимодедушкавыглянулвокноиахнулдаведьэтобиллэйбил